# Imports

In [17]:
%load_ext autoreload
%autoreload 2

import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

from sklearn.datasets import *

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import copy

from data_structures.tree_classifier import TreeClassifier
import utils.utils
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Prepare Data

In [1]:
# Download all datasets from sklearn
for m in [fetch_olivetti_faces, fetch_20newsgroups_vectorized, fetch_lfw_people, fetch_lfw_pairs, fetch_covtype, fetch_rcv1, fetch_kddcup99, fetch_california_housing]:
    print(m)
    try:
        all_ = m()
        train = m(subset='train')
        test = m(subset='test')
    except:
        pass

<function fetch_olivetti_faces at 0x7fb910b9f940>
<function fetch_20newsgroups_vectorized at 0x7fb910b71e50>
<function fetch_lfw_people at 0x7fb910af15e0>
<function fetch_lfw_pairs at 0x7fb910af1700>
<function fetch_covtype at 0x7fb910a9be50>
<function fetch_rcv1 at 0x7fb9009f19d0>
<function fetch_kddcup99 at 0x7fb910af1040>
<function fetch_california_housing at 0x7fb9009f1820>


In [12]:
# Download the data from two categories
cats = ['alt.atheism', 'sci.space']
ng_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=cats)
ng_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=cats)


vectorizer = TfidfVectorizer()
trans = vectorizer.fit(ng_train.data)
train_vectors = vectorizer.transform(ng_train.data)
test_vectors = vectorizer.transform(ng_test.data)
print("Number of datapoints: ", len(ng_train.data))
print("Number of features: ", train_vectors.shape[1])
print("Balance: ", np.sum(ng_train.target) / len(ng_train.target)) # 55-45, roughly balanced

N_COMPONENTS=100
pca = PCA(n_components=N_COMPONENTS)
pca.fit(train_vectors.toarray())
pca_train_vecs = pca.transform(train_vectors.toarray())
pca_test_vecs = pca.transform(test_vectors.toarray())

classes_arr = np.unique(ng_train.target)
classes = utils.utils.class_to_idx(classes_arr)

Number of datapoints:  1073
Number of features:  18217
Balance:  0.5526561043802423


# Compare our implementation's accuracy to sklearn

In [13]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(pca_train_vecs,ng_train.target)
print("sklearn Decision Tree Accuracy:", np.mean(dt.predict(pca_test_vecs) == ng_test.target))

#cross_val_score(dt, pca_train_vecs, ng_train.target, cv=10).mean()

sklearn Decision Tree Accuracy: 0.7812061711079944


In [14]:
rf = RandomForestClassifier(random_state=0)
rf.fit(pca_train_vecs,ng_train.target)
print("sklearn Random Forest Accuracy:", np.mean(rf.predict(pca_test_vecs) == ng_test.target))

#cross_val_score(rf, pca_train_vecs, ng_train.target, cv=10).mean()

sklearn Random Forest Accuracy: 0.8022440392706872


In [15]:
tc = TreeClassifier(data=pca_train_vecs, labels=ng_train.target, max_depth=5, classes=classes, verbose=False)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)

Train accuracy: 0.8657968313140727
Test accuracy: 0.7601683029453016
Num queries: 7545
Runtime: 8.689141988754272


In [16]:
tc = TreeClassifier(data=pca_train_vecs, labels=ng_train.target, max_depth=5, classes=classes, solver="EXACT", verbose=False)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)

Train accuracy: 0.869524697110904
Test accuracy: 0.7938288920056101
Num queries: 5300
Runtime: 5.325762748718262


# Make the dataset huge

In [3]:
doublings = 4
pca_train_vecs_huge = copy.deepcopy(pca_train_vecs)
pca_train_labels_huge = copy.deepcopy(ng_train.target)
print(pca_train_vecs_huge.shape)
for i in range(doublings):
    pca_train_vecs_huge = np.concatenate((pca_train_vecs_huge, pca_train_vecs_huge))
    pca_train_labels_huge = np.concatenate((pca_train_labels_huge, pca_train_labels_huge))
print(pca_train_vecs_huge.shape)

(1073, 100)
(17168, 100)


In [8]:
tc = TreeClassifier(data=pca_train_vecs_huge, labels=pca_train_labels_huge, max_depth=2, classes=classes, verbose=True, random_state=0)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)
tc.tree_print()

Calculated split with 100 queries
Calculated split with 5300 queries
Calculated split with 10800 queries
Fitting finished
Train accuracy: 0.8070829450139795
Test accuracy: 0.761570827489481
Num queries: 16200
Runtime: 6.089082956314087
|--- feature_1 <= -0.037112277192202114
|   |--- feature_3 <= 0.11134765675972713
|   |   |--- class: 0
|   |--- feature_3 > 0.11134765675972713
|   |   |--- class: 1
|--- feature_1 > -0.037112277192202114
|   |--- feature_3 <= -0.07697109870758101
|   |   |--- class: 0
|   |--- feature_3 > -0.07697109870758101
|   |   |--- class: 1




In [9]:
# We should implement vanilla TreeClassifier --> which uses identity bins

tc = TreeClassifier(data=pca_train_vecs_huge, labels=pca_train_labels_huge, max_depth=2, classes=classes, solver="EXACT", verbose=True, random_state=0)
start = time.time()
tc.fit()
end = time.time()
print("Train accuracy:", np.mean(tc.predict_batch(pca_train_vecs)[0] == ng_train.target))
print("Test accuracy:", np.mean(tc.predict_batch(pca_test_vecs)[0] == ng_test.target))
print("Num queries:", tc.num_queries)
print("Runtime:", end-start)
tc.tree_print()

Calculated split with 17168 queries
Calculated split with 5728 queries
Calculated split with 11440 queries
Fitting finished
Train accuracy: 0.8070829450139795
Test accuracy: 0.761570827489481
Num queries: 34336
Runtime: 20.192893028259277
|--- feature_1 <= -0.037112277192202114
|   |--- feature_3 <= 0.11134765675972713
|   |   |--- class: 0
|   |--- feature_3 > 0.11134765675972713
|   |   |--- class: 1
|--- feature_1 > -0.037112277192202114
|   |--- feature_3 <= -0.07697109870758101
|   |   |--- class: 0
|   |--- feature_3 > -0.07697109870758101
|   |   |--- class: 1




# Verify our implementation of baseline models agrees with sklearn's in terms of accuracy

# Classification:

In [42]:
from data_structures.wrappers.random_forest_classifier import RandomForestClassifier as RFC_ours
from data_structures.wrappers.extremely_random_forest_classifier import ExtremelyRandomForestClassifier as ERFC_ours

from data_structures.wrappers.random_forest_regressor import RandomForestRegressor as RFR_ours
from data_structures.wrappers.extremely_random_forest_regressor import ExtremelyRandomForestRegressor as ERFR_ours

from sklearn.ensemble import RandomForestClassifier as RFC_sklearn
from sklearn.ensemble import ExtraTreesClassifier as ERFC_sklearn

from sklearn.ensemble import RandomForestRegressor as RFR_sklearn
from sklearn.ensemble import ExtraTreesRegressor as ERFR_sklearn

from utils.constants import GINI, BEST, EXACT, MSE

# TODO(@motiwari): Allow for gradient boosted comparisons as well
def compare_accuracies(
    compare: str = "RFC",
    train_data: np.ndarray = None,
    train_targets: np.ndarray = None,
    test_data: np.ndarray = None,
    test_targets: np.ndarray = None,
    num_seeds: int = 10,
) -> bool:
    our_train_accs = []
    our_test_accs = []
    their_train_accs = []
    their_test_accs = []
    for seed in range(num_seeds):
        # Ok to have n_jobs = -1 throughout?
        if compare == "RFC":
            our_model = RFC_ours(data=train_data, labels=train_labels, n_estimators=5, max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=GINI, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            their_model = RFC_sklearn(n_estimators=5, criterion='gini', max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, n_jobs=-1, random_state=seed, verbose=0)
        elif compare == "ERFC":
            our_model = ERFC_ours(data=train_data, labels=train_labels, n_estimators=5, max_depth=5, num_bins=None, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=GINI, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            their_model = ERFC_sklearn(n_estimators=5, criterion='gini', max_depth=5, min_samples_split=2, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=False, n_jobs=-1, random_state=seed, verbose=0)
        elif compare == "RFR":
            our_model = RFR_ours(data=train_data, labels=train_labels, n_estimators=5, max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=MSE, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            their_model = RFR_sklearn(n_estimators=5, criterion='squared_error', max_depth=5, min_samples_split=2, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, n_jobs=-1, random_state=seed, verbose=0)    
        elif compare == "ERFR":
            our_model = ERFR_ours(data=train_data, labels=train_labels, n_estimators=5, max_depth=5, num_bins=None, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=MSE, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            their_model = ERFR_sklearn(n_estimators=5, criterion='squared_error', max_depth=5, min_samples_split=2, max_features='auto', min_impurity_decrease=0.0, bootstrap=False, n_jobs=-1, random_state=seed, verbose=0)
        else:
            raise NotImplementedError("Need to decide what models to compare")
        
        our_model.fit()
        their_model.fit(pca_train_vecs_huge, pca_train_labels_huge)

        if compare == "RFC" or compare == "ERFC" or compare == "HRFC":
            our_train_acc = np.mean(our_model.predict_batch(train_data)[0] == train_targets)
            our_test_acc = np.mean(our_model.predict_batch(test_data)[0] == test_targets)
            their_train_acc = np.mean(their_model.predict(train_data) == train_targets)
            their_test_acc = np.mean(their_model.predict(test_data) == test_targets)
        elif compare == "RFR" or compare == "ERFR" or compare == "HRFR":
            our_train_acc = np.mean((our_model.predict_batch(train_data) - train_targets)**2)
            our_test_acc = np.mean((our_model.predict_batch(test_data) - test_targets)**2)
            their_train_acc = np.mean((our_model.predict_batch(train_data) - train_targets)**2)
            their_test_acc = np.mean((our_model.predict_batch(test_data) - test_targets)**2)

        print("(Ours) Train accuracy:", our_train_acc)
        print("(Ours) Test accuracy:", our_test_acc)
        print("(Theirs) Train accuracy:", their_train_acc)
        print("(Theirs) Test accuracy:", their_test_acc)
        print("-" * 30)

        our_train_accs.append(our_train_acc)
        our_test_accs.append(our_test_acc)
        their_train_accs.append(their_train_acc)
        their_test_accs.append(their_test_acc)
    
    our_avg_train = np.mean(our_train_accs)
    our_std_train = np.std(our_train_accs)

    our_avg_test = np.mean(our_test_accs)
    our_std_test = np.std(our_test_accs)
    
    their_avg_train = np.mean(their_train_accs)
    their_std_train = np.std(their_train_accs)
    
    their_avg_test = np.mean(their_test_accs)
    their_std_test = np.std(their_test_accs)
    
    # See if confidence intervals overlap
    overlap = np.abs(their_avg_test - our_avg_test) < their_std_test + our_std_test
    return overlap, our_avg_train, our_std_train, our_avg_test, our_std_test, their_avg_train, their_std_train, their_avg_test, their_std_test

In [69]:
train_data = pca_train_vecs
train_labels = ng_train.target
test_data = pca_test_vecs
test_labels = ng_test.target

compare_accuracies("RFC", train_data, train_labels, test_data, test_labels)

(Ours) Train accuracy: 0.8648648648648649
(Ours) Test accuracy: 0.7896213183730715
(Theirs) Train accuracy: 0.7931034482758621
(Theirs) Test accuracy: 0.7475455820476858
------------------------------
(Ours) Train accuracy: 0.8704566635601119
(Ours) Test accuracy: 0.7643758765778401
(Theirs) Train accuracy: 0.8443616029822927
(Theirs) Test accuracy: 0.7643758765778401
------------------------------
(Ours) Train accuracy: 0.875116495806151
(Ours) Test accuracy: 0.7601683029453016
(Theirs) Train accuracy: 0.7996272134203168
(Theirs) Test accuracy: 0.7545582047685835
------------------------------
(Ours) Train accuracy: 0.8564771668219944
(Ours) Test accuracy: 0.726507713884993
(Theirs) Train accuracy: 0.8285181733457595
(Theirs) Test accuracy: 0.7769985974754559
------------------------------
(Ours) Train accuracy: 0.8499534016775396
(Ours) Test accuracy: 0.729312762973352
(Theirs) Train accuracy: 0.8387698042870456
(Theirs) Test accuracy: 0.7629733520336606
-----------------------------

(True,
 0.8659832246039143,
 0.009922707663900228,
 0.7559607293127629,
 0.018232819074333793,
 0.8183597390493942,
 0.01963531888120251,
 0.7612903225806451,
 0.01223978918893207)

In [70]:
compare_accuracies("ERFC", train_data, train_labels, test_data, test_labels)

(Ours) Train accuracy: 0.7185461323392358
(Ours) Test accuracy: 0.6367461430575035
(Theirs) Train accuracy: 0.777260018639329
(Theirs) Test accuracy: 0.7307152875175316
------------------------------
(Ours) Train accuracy: 0.7315936626281454
(Ours) Test accuracy: 0.638148667601683
(Theirs) Train accuracy: 0.700838769804287
(Theirs) Test accuracy: 0.6900420757363254
------------------------------
(Ours) Train accuracy: 0.7073625349487418
(Ours) Test accuracy: 0.5974754558204769
(Theirs) Train accuracy: 0.7092264678471575
(Theirs) Test accuracy: 0.6774193548387096
------------------------------
(Ours) Train accuracy: 0.7073625349487418
(Ours) Test accuracy: 0.6199158485273493
(Theirs) Train accuracy: 0.6952469711090401
(Theirs) Test accuracy: 0.6605890603085554
------------------------------
(Ours) Train accuracy: 0.700838769804287
(Ours) Test accuracy: 0.6255259467040674
(Theirs) Train accuracy: 0.6421248835041938
(Theirs) Test accuracy: 0.6143057503506312
------------------------------

(True,
 0.7089468779123951,
 0.0210387168543773,
 0.6175315568022441,
 0.016865904357043132,
 0.6953401677539609,
 0.048951665274391376,
 0.6605890603085554,
 0.04692487467869127)

# Regression: 

### Weirdly, we get exactly the same results as sklearn for all seeds. We should investigate whether this is a bug

In [73]:
np.random.seed(0)

data, targets = fetch_california_housing(return_X_y=True)
random_idcs = np.random.choice(len(data), size=len(data),replace=False)
data = data[random_idcs]
targets = targets[random_idcs]

TRAIN_TEST_SPLIT = 16000
train_data = data[:TRAIN_TEST_SPLIT]
train_targets = targets[:TRAIN_TEST_SPLIT]

test_data = data[TRAIN_TEST_SPLIT:]
test_targets = targets[TRAIN_TEST_SPLIT:]

In [74]:
compare_accuracies("RFR", train_data, train_targets, test_data, test_targets)

(Ours) Train accuracy: 3.6047103904415625
(Ours) Test accuracy: 3.557280888823238
(Theirs) Train accuracy: 3.6047103904415625
(Theirs) Test accuracy: 3.557280888823238
------------------------------
(Ours) Train accuracy: 3.698837202030057
(Ours) Test accuracy: 3.649268394695702
(Theirs) Train accuracy: 3.698837202030057
(Theirs) Test accuracy: 3.649268394695702
------------------------------
(Ours) Train accuracy: 3.668971394724753
(Ours) Test accuracy: 3.6183697080805217
(Theirs) Train accuracy: 3.668971394724753
(Theirs) Test accuracy: 3.6183697080805217
------------------------------
(Ours) Train accuracy: 3.732563296199221
(Ours) Test accuracy: 3.6927617072229575
(Theirs) Train accuracy: 3.732563296199221
(Theirs) Test accuracy: 3.6927617072229575
------------------------------
(Ours) Train accuracy: 3.6404918646836975
(Ours) Test accuracy: 3.5838844263183107
(Theirs) Train accuracy: 3.6404918646836975
(Theirs) Test accuracy: 3.5838844263183107
------------------------------
(Ours

(True,
 3.6764456283237954,
 0.038867352099088585,
 3.632019514986588,
 0.04266395398168999,
 3.6764456283237954,
 0.038867352099088585,
 3.632019514986588,
 0.04266395398168999)

In [76]:
compare_accuracies("ERFR", train_data, train_targets, test_data, test_targets)

(Ours) Train accuracy: 3.6815378247519694
(Ours) Test accuracy: 3.6407684028645155
(Theirs) Train accuracy: 3.6815378247519694
(Theirs) Test accuracy: 3.6407684028645155
------------------------------
(Ours) Train accuracy: 3.6383368678729986
(Ours) Test accuracy: 3.5958683000240095
(Theirs) Train accuracy: 3.6383368678729986
(Theirs) Test accuracy: 3.5958683000240095
------------------------------
(Ours) Train accuracy: 3.6733018758052713
(Ours) Test accuracy: 3.627187867032216
(Theirs) Train accuracy: 3.6733018758052713
(Theirs) Test accuracy: 3.627187867032216
------------------------------
(Ours) Train accuracy: 3.6589668229880212
(Ours) Test accuracy: 3.618059136379204
(Theirs) Train accuracy: 3.6589668229880212
(Theirs) Test accuracy: 3.618059136379204
------------------------------
(Ours) Train accuracy: 3.6114095412271663
(Ours) Test accuracy: 3.565551845832376
(Theirs) Train accuracy: 3.6114095412271663
(Theirs) Test accuracy: 3.565551845832376
------------------------------
(

(True,
 3.6584509634706954,
 0.020607753150392236,
 3.6148928935595626,
 0.022030467087038482,
 3.6584509634706954,
 0.020607753150392236,
 3.6148928935595626,
 0.022030467087038482)

# Verify MAB solvers are faster in speed but have no change in accuracy

# Classification

In [21]:
import time
from typing import Any, Tuple

# Classification #
# Vanilla random forest + H + GB + GBH
from data_structures.wrappers.random_forest_classifier import RandomForestClassifier as RFC
from data_structures.wrappers.histogram_random_forest_classifier import HistogramRandomForestClassifier as HRFC
from data_structures.wrappers.gradient_boosted_random_forest_classifier import GradientBoostedRandomForestClassifier as GBRFC
from data_structures.wrappers.gradient_boosted_histogram_random_forest_classifier import GradientBoostedHistogramRandomForestClassifier as GBHRFC

# Extremely random forest + GB (already histogrammed)
from data_structures.wrappers.extremely_random_forest_classifier import ExtremelyRandomForestClassifier as ERFC
from data_structures.wrappers.gradient_boosted_extremely_random_forest_classifier import GradientBoostedExtremelyRandomForestClassifier as GBERFC

# Random patches + H + GB + GBH
from data_structures.wrappers.random_patches_classifier import RandomPatchesClassifier as RPC
from data_structures.wrappers.histogram_random_patches_classifier import HistogramRandomPatchesClassifier as HRPC
from data_structures.wrappers.gradient_boosted_random_patches_classifier import GradientBoostedRandomPatchesClassifier as GBRPC
from data_structures.wrappers.histogram_random_patches_classifier import HistogramRandomPatchesClassifier as HBRPC


# Regression #
# Vanilla random forest + H + GB + GBH
from data_structures.wrappers.random_forest_regressor import RandomForestRegressor as RFR
from data_structures.wrappers.histogram_random_forest_regressor import HistogramRandomForestRegressor as HRFR
from data_structures.wrappers.gradient_boosted_random_forest_regressor import GradientBoostedRandomForestRegressor as GBRFR
from data_structures.wrappers.gradient_boosted_histogram_random_forest_regressor import GradientBoostedHistogramRandomForestRegressor as GBHRFR

# Extremely random forest + GB (already histogrammed)
from data_structures.wrappers.extremely_random_forest_regressor import ExtremelyRandomForestRegressor as ERFR
from data_structures.wrappers.gradient_boosted_extremely_random_forest_regressor import GradientBoostedExtremelyRandomForestRegressor as GBERFR

# Random patches + H + GB + GBH
from data_structures.wrappers.random_patches_regressor import RandomPatchesRegressor as RPR
from data_structures.wrappers.histogram_random_patches_regressor import HistogramRandomPatchesRegressor as HRPR
from data_structures.wrappers.gradient_boosted_random_patches_regressor import GradientBoostedRandomPatchesRegressor as GBRPR
from data_structures.wrappers.histogram_random_patches_regressor import HistogramRandomPatchesRegressor as HBRPR

In [38]:
def time_measured_fit(
    model: Any,
    data: np.ndarray = None,
    labels: np.ndarray = None,
) -> float:
    """
    Returns wall clock time of training the model, in seconds.
    
    Has a side effect: trains the model.
    """
    assert (data is None and labels is None) or (data is not None and labels is not None), "Need to pass both data and labels at the same time"
    
    if data is None:
        start = time.time()
        model.fit()
        end = time.time()
    else:
        start = time.time()
        model.fit(data, labels)
        end = time.time()
    
    return end - start

In [45]:
from utils.constants import GINI, BEST, EXACT, MAB, MSE


def compare_runtimes(
    compare: str = "HRFC",
    train_data: np.ndarray = None,
    train_targets: np.ndarray = None,
    test_data: np.ndarray = None,
    test_targets: np.ndarray = None,
    num_seeds: int = 10,
) -> bool:
    # Runtimes
    our_train_times = []
    their_train_times = []
    
    # For accuracies
    our_train_accs = []
    our_test_accs = []
    their_train_accs = []
    their_test_accs = []
    for seed in range(num_seeds):
        # Ok to have n_jobs = -1 throughout?
        if compare == "HRFC":
            their_model = HRFC(data=train_data, labels=train_labels, n_estimators=5, max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=GINI, splitter=BEST, solver=EXACT, random_state=seed, verbose=False)
            our_model = HRFC(data=train_data, labels=train_labels, n_estimators=5, max_depth=5, min_samples_split=2, min_impurity_decrease=0, max_leaf_nodes=None, budget=None, criterion=GINI, splitter=BEST, solver=MAB, random_state=seed, verbose=False)
        else:
            raise NotImplementedError("Need to decide what models to compare")
        
        assert 'sklearn' not in their_model.__module__, "Cannot use sklearn models for runtime comparisons"
        
        our_runtime = time_measured_fit(our_model)
        print("HERE")
        their_runtime = time_measured_fit(their_model, train_data, train_labels)
        print("HERE")

        if compare == "RFC" or compare == "ERFC" or compare == "HRFC":
            our_train_acc = np.mean(our_model.predict_batch(train_data)[0] == train_targets)
            our_test_acc = np.mean(our_model.predict_batch(test_data)[0] == test_targets)
            their_train_acc = np.mean(their_model.predict_batch(train_data) == train_targets)
            their_test_acc = np.mean(their_model.predict_batch(test_data) == test_targets)
        elif compare == "RFR" or compare == "ERFR":
            our_train_acc = np.mean((our_model.predict_batch(train_data) - train_targets)**2)
            our_test_acc = np.mean((our_model.predict_batch(test_data) - test_targets)**2)
            their_train_acc = np.mean((our_model.predict_batch(train_data) - train_targets)**2)
            their_test_acc = np.mean((our_model.predict_batch(test_data) - test_targets)**2)

        print("(Ours) Train accuracy:", our_train_acc)
        print("(Ours) Test accuracy:", our_test_acc)
        print("(Theirs) Train accuracy:", their_train_acc)
        print("(Theirs) Test accuracy:", their_test_acc)
        print("*" * 30)
        print("(Ours) Runtime:", our_runtime)
        print("(Theirs) Runtime:", their_runtime)
        print("-" * 30)

        our_train_accs.append(our_train_acc)
        our_test_accs.append(our_test_acc)
        their_train_accs.append(their_train_acc)
        their_test_accs.append(their_test_acc)
    
    # For accuracies
    our_avg_train = np.mean(our_train_accs)
    our_std_train = np.std(our_train_accs)

    our_avg_test = np.mean(our_test_accs)
    our_std_test = np.std(our_test_accs)
    
    their_avg_train = np.mean(their_train_accs)
    their_std_train = np.std(their_train_accs)
    
    their_avg_test = np.mean(their_test_accs)
    their_std_test = np.std(their_test_accs)
    
    # For runtimes
    our_avg_train_time = np.mean(our_train_times)
    our_std_train_time = np.std(our_train_times)
    
    their_avg_train_time = np.mean(their_train_times)
    their_std_train_time = np.std(their_train_times)
    
    
    # See if confidence intervals overlap
    overlap = np.abs(their_avg_test - our_avg_test) < their_std_test + our_std_test
    return overlap, our_avg_train, our_std_train, our_avg_test, our_std_test, their_avg_train, their_std_train, their_avg_test, their_std_test, our_avg_train_time, our_std_train_time, their_avg_train_time, their_std_train_time

In [47]:
train_data = pca_train_vecs
train_labels = ng_train.target
test_data = pca_test_vecs
test_labels = ng_test.target

compare_runtimes("HRFC", train_data, train_labels, test_data, test_labels)

HERE
HERE
-0.010757071124726464
-0.016373522067245752
0.014873883406697987
0.11164511828873312
-0.09070751723835649
0.06421724768525189
-0.10150580252584815
0.038397116225528695
0.0045851787234224865
-0.04845768542690424
0.11164511828873312
-0.14123045616665775
0.016775099887909147
-0.015583580700666486
-0.016575497174043705
-0.10150580252584815
0.06421724768525189
0.01973693387780641
0.012414594286964006
0.11164511828873312
-0.018711146571476673
0.012414594286964006
-2.5892150008706427e-05
-0.016373522067245752
0.017753436420089464
0.05963419641782859
-0.021195073334473485
-0.028146834808715437
0.09781528530379549
0.04454504681584671
-0.0071574916471595935
0.04454504681584671
0.02864686570296453
-0.02919528716513019
0.04587921020066851
0.09781528530379549
0.0334991251792923
0.0334991251792923
-0.015866333052576946
0.01526722926216234
0.067984221329163
-0.021195073334473485
0.0334991251792923
0.009856112879239156
0.09781528530379549
0.04778319733556065
0.015185547532139028
-0.044186635

0.04299464867016174
0.0125652663037425
0.02770039173266097
-0.07410894196014595
0.014447399528649693
0.0931909480087751
-0.0652134500325176
0.07739162308842011
0.06995441046351773
-0.020616715084792316
0.0125652663037425
0.02631242280013488
-0.05926668658833357
-0.0652134500325176
-0.013071993947350405
-0.05926668658833357
0.013628813147995298
-0.01245119776966133
-0.033346539935946294
0.04972198697623664
-0.024408157595629375
-0.09266274961921903
-0.04184827297753426
-0.03340300288471668
-0.05590593912776228
0.006737224733480784
-0.05903080921192342
0.05653709859559769
-0.02911200861460511
0.007745424167273033
-0.06770243276992464
0.038466750305928575
0.027044654037775305
-0.022944941736086737
0.011080142092223504
-0.024408157595629375
-0.06770243276992464
-0.07039504417863512
-0.013868526587646902
-0.06761417158686688
0.007745424167273033
0.0780998208880776
0.0402088958196807
-0.005620498583278156
-0.06308310184314576
-0.0222123929197221
0.024712291652342483
-0.11035488663066668
-0.0

-0.057072967509093714
-0.03813336113092907
-0.05555977652241356
0.0733819120859587
0.00846064581792935
-0.07021007663369014
-0.018883702083611547
-0.014273160646467213
0.04501717771478556
0.026300893039058124
-0.03765496746005188
0.1373579815324449
0.01718492265269384
0.046467908459617804
-0.05623390897396269
0.06890884491239185
0.026300893039058124
0.09828867264706542
0.09828867264706542
0.010162546941618115
0.010044861753152516
0.01718492265269384
-0.014273160646467213
0.09828867264706542
0.013806609544467193
0.026300893039058124
0.009100933830494802
-0.05623390897396269
-0.020340631948073243
-0.014273160646467213
0.010162546941618115
-0.03985694989948699
-0.03314107191856847
0.030291001613860107
-0.06600765124856667
-0.034957031567339515
0.03990958239913202
-0.07151269325959557
0.10198232024022134
-0.0048537310772352885
0.0010380482923051859
-0.06600765124856667
-0.04610225057006296
-0.07151269325959557
-0.0048537310772352885
0.0010380482923051859
-0.07151269325959557
0.039909582399

-0.011275371630576916
0.08526904863826978
0.0913627422111281
-0.04730387045085477
0.007853295437862538
-0.05838302714737064
-0.004052480300581162
-0.011275371630576916
0.19246845448947053
0.19246845448947053
-0.007509384579766697
-0.017314013391325253
-0.04730387045085477
0.0913627422111281
-0.03309464814235578
0.0913627422111281
0.012245761615541145
0.0262865081671647
-0.05293236022581434
-0.04337335686528804
-0.004138496679322251
-0.007509384579766697
0.0479992409490103
-0.05373837536454436
-0.0035121919031466937
-0.0029994671132437434
-0.016414123570528877
0.01526595572409123
0.06926913020794266
-0.016385206581592874
0.07622873788435433
-0.015216853177162471
-0.012691852175965482
-0.08811217287663094
-0.03117307060241699
0.01614537020918152
0.011897727309408263
-0.0759737275227528
0.01526595572409123
-0.0035642712283702658
-0.013482306177064918
-0.012691852175965482
0.04838809423209246
-0.013482306177064918
-0.016785523646084342
-0.05373837536454436
0.004517354390426198
-0.045623103

0.10143786755459093
-0.034990185228595504
-0.050283258593456365
0.0022040159859735564
0.006167887275645615
0.07330451098429741
0.04653741470355073
0.006167887275645615
0.02008043894141251
0.03375050080739018
0.03034440286019911
0.01806769346764487
0.7627203815260257
-0.0752055818784175
-0.007228189106577593
0.03442114369151054
-0.09451900327703515
0.07963084600865643
0.02580929447647579
0.00662609661639133
0.014928183454442998
0.03135768295138004
-0.11756595319164852
0.005248728885490873
-0.006830706335161274
0.02707428920805527
0.08031677127779502
0.7627203815260257
-0.11756595319164852
-0.013846385682158608
-0.08319484858872576
-0.008219664093482192
-0.09233664266849273
-0.017055084615020796
0.7627203815260257
0.006554468287238616
-0.017209764270377084
-0.013151851760824453
-0.006251961589924945
0.005650102083096029
0.018301374298145885
-0.1664282644346299
0.018301374298145885
-0.02546769744810646
-0.0006800554992976132
0.06170657428944491
0.005650102083096029
0.02033503265463887
0.0

...

-0.05995229279026397
0.06002436981622436
9.83984543230375e-05
-0.01555569099197572
-0.050354795054081976
0.009888791881377618
-0.01555569099197572
-0.006493699458168986
-0.01814678845771147
-0.014989909702976114
0.028061221724072525
0.07161231105282195
-0.0531978547047838
-0.006635658810039031
0.02105548642373589
0.2291233126045335
-0.05639834861179524
-0.030923680663143372
-0.01954011746836866
-0.006635658810039031
-0.08388656528156302
-0.002076797881059732
-0.05639834861179524
-0.01954011746836866
-0.006635658810039031
-0.05639834861179524
0.2291233126045335
0.011949261857712484
0.002841066336723287
0.004635867118437069
0.00982368856771693
0.08362504087830878
-0.004847775756653353
-0.05639834861179524
0.003223655951952877
-0.022312867868068048
-0.012711693180592454
0.020183118387499553
-0.017534143464751226
-0.03524875218429078
-0.018133759688188904
-0.03524875218429078
-0.054458400969558625
-0.012711693180592454
-0.018133759688188904
-0.017534143464751226
-0.002952312614118442
-0.00

-0.0255837296939258
-0.11258637174117334
-0.020789702179478717
-0.006647772369549028
-0.021533428684765602
0.04836242987442829
-0.006647772369549028
-0.03503902552391424
-0.013184967604430779
-0.03704333714950526
0.06111280342487288
-0.027939490080400985
-0.06403374242608488
0.016276525742963593
-0.04307155024172061
-0.12104568581995515
0.0304813014724118
-0.0960803410836303
-0.0671048658802205
-0.0633765994766309
-0.06532560203578572
-0.0075531277112208675
0.11587907704393263
0.006892424844182317
-0.011192704248345758
0.11587907704393263
-0.027939490080400985
-0.010590292198037497
-0.01842550463261981
-0.06532560203578572
0.026913163742057402
0.12050942070527498
-0.010590292198037497
0.11587907704393263
-0.0010464650545320337
-0.04463808187951015
-0.001930591748417083
-0.07057150947742946
0.012376021654386282
0.03911023919144627
-0.016839386682779525
0.02484484298166253
-0.014395441748744148
0.004413135544312887
0.04053632361579138
-0.010093578947924089
-0.07930447613481711
0.02121195

0.00045965850584604707
-0.11260193072164608
0.03092204638812616
-0.03116612062343768
-0.11260193072164608
0.0237505266944534
0.10131712979984467
-0.005780929853960265
0.012626910714153838
-0.02087375805771991
-0.03116612062343768
0.007550230131663292
-0.004368344508046403
0.029081990553141906
-0.018220856309400593
0.06342641677306564
0.026617654587889006
-0.035780446779582864
0.026617654587889006
0.014059133178232918
-0.014947750334398548
0.03432380175178426
0.06342641677306564
-0.1173690909546992
0.0337130852247844
-0.006998058058582092
-0.0009380578562993274
0.02219297780962594
0.029081990553141906
-0.1173690909546992
0.024016321400652525
0.0356483081697955
-0.031118245731926057
-0.041445148758973
0.017980026412695384
0.029081990553141906
-0.012222551560104676
0.0331750651687692
-0.012397088928016154
-0.05500595779401844
0.0007837361903182962
-0.005224918322864858
-0.021331439410825343
0.018094310714869517
0.008248215910032183
-0.02037210210352662
0.01958306968714381
-0.0273934366478

0.008169860208259008
0.14210754086364052
0.03726470774982612
-0.009858115505243278
-0.07612792085929854
-0.03340407309591471
0.0612840852760974
-0.03340407309591471
-0.03340407309591471
-0.03179995059937638
-0.017125386300676837
0.03726470774982612
-0.04902453549419593
-0.03340407309591471
-0.017125386300676837
0.0612840852760974
0.006746748360624876
-0.07612792085929854
-0.021823458919231258
-0.04902453549419593
0.0225345281913647
-0.001096691055161508
-0.013630595976937544
0.013134410714407546
-0.006439678607081561
-0.017522755802505247
-0.24625836649949534
-0.017522755802505247
-0.000877182568294165
-0.013630595976937544
-0.24625836649949534
-0.006439678607081561
0.0010312006628408095
0.0010312006628408095
-0.001565815907740381
0.00021964377201631378
-0.03207245042439542
-0.24625836649949534
-0.006256167685130572
-0.003204543756090939
-0.006439678607081561
-0.0032298409773500974
-0.003204543756090939
0.002121798643153393
-0.013630595976937544
-0.006438045530679194
0.0150613173948980

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()